In [1]:
from IPython.display import display
import wordle_functions as wf

In [2]:
#testcases for generateColourPattern
assert wf.generateColourPattern("apple","hello") == [0,0,0,2,1]
assert wf.generateColourPattern("hello","hello") == [2,2,2,2,2]
assert wf.generateColourPattern("hello","apple") == [0,1,0,2,0]
assert wf.generateColourPattern("lllll","apple") == [0,0,0,2,0]
assert wf.generateColourPattern("llple","aaall") == [1,0,0,2,0]

In [3]:
def generateFirstTurnEntropy() :
    """Dedicated function for generating the first guess entropy dictionary, with a progress display"""

    dh = display('Starting...',display_id=True)
    wordEntropies = {}
    for guess in wf.loadGuessableWords() :
        dh.update('Evaluating ' + guess)
        patternDist = {}
        for answer in wf.loadAnswers() :
            pattern = ''.join(str(x) for x in wf.generateColourPattern(guess,answer))
            patternDist[pattern] = patternDist.get(pattern,0) + 1 #increment the count for this pattern
        e = wf.calculateEntropy(patternDist)
        wordEntropies[guess] = wordEntropies.get(guess,0) + e
    dh.update('Complete')
    return wordEntropies

In [4]:
d = generateFirstTurnEntropy()

'Complete'

In [5]:
max(d, key=d.get)

'soare'

In [6]:
d = dict(sorted(d.items(), reverse=True, key=lambda item: item[1]))

In [7]:
with open('first_turn_entropies.txt', 'w') as file:
    for key, value in d.items():
        file.write(f'{key}: {value}\n')